In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np
import csv
import subprocess
import os
import matplotlib.pyplot as plt
import subprocess
%matplotlib inline

# 1. Questaal(DFT) calculations
#3 files ctrl.fept, script.sh, and syml.fept used
#Output of DFT generates spf.fept text file, which should be pre-processed for plotting spectral-function.

In [ ]:
command = ['sbatch', 'script.sh']
result = subprocess.run(command, capture_output=True, text=True)

# Check if the command was executed successfully
if result.returncode == 0:
    print("Job submitted successfully.")
    job_id = result.stdout.strip().split()[-1]  # Assuming Slurm returns "Submitted batch job <job_id>"
    print("Job ID:", job_id)
else:
    print("Error submitting job.")
    print("Error:", result.stderr)
    exit()  # Exit if job submission failed
    
file_path = os.path.join(os.getcwd(), 'spf.fept')
last_size = -1

while True:
    if os.path.exists(file_path):
        current_size = os.path.getsize(file_path)
        if current_size == last_size:
            print("File size stable, assuming DFT run is complete.")
            break
        else:
            last_size = current_size
    else:
        print("Waiting for DFT run to start and create spf.fept...")
    time.sleep(60)  # Check every 60 seconds

# 2. Data-Preprocessing: Part1
# (a) Preprocessing to get symmetry points

In [ ]:
# Get the current working directory path
cwd = os.getcwd()

# Check if the file exists in the current directory
if os.path.exists(os.path.join(cwd, 'spf.fept')):
    with open('spf.fept', 'r') as file:
        lines = file.readlines()

    with open('first_line.dat', 'w') as file:
        file.write(lines[0])
    with open('first_line.dat','r') as f:
        content = f.read()
        content = content.replace('#', '').strip()
        file_string = content.split()
        syml_val  = [0] + [float(num) for num in file_string]

#  2. Data-Preprocessing: Part1
# (b) converting spf.fept files to csv for later conversion to dataframe

In [ ]:
input_file_path = 'spf.fept'

with open(input_file_path, 'r') as file:
    lines = file.readlines()
with open(input_file_path, 'w') as file:
    file.writelines(lines[1:])

# Path to the new CSV file
output_file_path = 'output.csv'

# Open the original file and read the lines
with open(input_file_path, 'r') as infile, open(output_file_path, 'w', newline='') as csvfile:
    # Assuming each line of the file needs to be converted
    writer = csv.writer(csvfile)
    
    # Process each line in the original file
    for line in infile:
        # Assuming the original file might have comments or empty lines
        if line.split() and not line.startswith('#'):
            values = line.strip().split()
              # Write the values as a row in the CSV file
            writer.writerow(values)

# 2. Data-Preprocessing: Part-2 :- Further preprocessing as a dataframe to plot Spectral function.

In [ ]:
df = pd.read_csv('output.csv', names= ['frequency','distance','A_up', 'A_down'])
df['A_up'] = (df['A_up'] + df['A_down'])/2
df['frequency']=df['frequency'].apply(lambda x: x*13.60569806)
df = df.drop(columns=['A_down'])
df = df.rename(columns={'A_up': 'A'})
df.head()

# 3. Data Visualization

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_spectral_function(df,filename=None):
    """
    Plot the spectral function.
    :param df: pandas DataFrame with the data.
    """
    fig, ax = plt.subplots(figsize=(8, 5))
    sc = ax.scatter(df['distance'], df['frequency'], c=df['A'], vmin=0, vmax=100,cmap='cividis', rasterized=True)
    cbar = fig.colorbar(sc, ax=ax)
    cbar.ax.tick_params(labelsize=15)
    array = syml_val
    labels = ['X','$\Gamma$','M','X','R','A','$\Gamma$','Z']
    ax.set_ylabel('Energy (eV)',fontsize=20)
    ax.set_xticks(array)
    ax.set_xticklabels(labels,fontsize=20)
    ax.grid(axis="x",color='white')
    plt.yticks(fontsize=20)
    ax.tick_params(axis='y', direction='in',size=15)
    ax.tick_params(axis='x', direction='in')
    ax.set_xlim(0,3.75804)
    ax.set_ylim(-3.5,2.5)
    plt.axhline(y=0, xmin=0, xmax=3.75804,color='white',linestyle="--")
    plt.tight_layout()
    plt.title("Spectral function of L1$_0$ phase FePt")
    if filename:
        plt.savefig(filename, format="png")
    plt.show()
    
plot_spectral_function(df,filename="fept.png")